In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pyrealsense2 as rs
import os
import numpy as np
import sys
import scipy
import random
import math
import skimage.io
import datetime
import open3d as o3d

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize

Stones_DIR = os.path.join(ROOT_DIR, "datasets/stones")


#from samples.blister import blister_mul_class
import stones

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
# Download the weight from our Github repository: https://github.com/HKUST-RML
STONES_MODEL_PATH = "./mask_rcnn_stones_0100.h5"


# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

from poke_grasp.msg import stone_pose
import rospy
import geometry_msgs.msg
import time
import actionlib
from std_msgs.msg import String

In [ ]:
rospy.init_node('stone_segmentation')

In [ ]:
def make_directories():
    if not os.path.exists("JPEGImages/"):
        os.makedirs("JPEGImages/")
    if not os.path.exists("depth/"):
        os.makedirs("depth/")

In [ ]:
# Start instance segmentation by Mask RCNN
class InferenceConfig(stones.StonesConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
##config.display()

# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(STONES_MODEL_PATH, by_name=True)

# Load dataset
# Get the dataset from the releases page
# https://github.com/matterport/Mask_RCNN/releases
dataset = stones.StonesDataset()
dataset.load_stones(Stones_DIR, "train")

# Must call before using the dataset
dataset.prepare()

print("Image Count: {}".format(len(dataset.image_ids)))
print("Class Count: {}".format(dataset.num_classes))
    
class_names = dataset.class_names

In [ ]:
def pixel_to_camera(pixel, intrin, depth):
    #depth = depth #/ 1000
    X = (pixel[0]-intrin[0]) * depth / intrin[2]
    Y = (pixel[1]-intrin[1]) * depth / intrin[3]
    return [X, Y]

In [ ]:
# Compute position of mask center and rotation of mask
def get_mask_pose(depth_intrin, depth_array, m_c, mm_pair, max_d, min_d, surf_normal):
    #position = m_c
    print('select mask center', m_c)
    m_c_dist = depth_array[m_c[1], m_c[0]]/1000
    #position = rs.rs2_deproject_pixel_to_point(depth_intrin, m_c, m_c_dist)
    position = pixel_to_camera(m_c, depth_intrin, m_c_dist)
    min2max_vec = [mm_pair[0][0]-mm_pair[1][0], mm_pair[0][1]-mm_pair[1][1]]
    yaw = math.atan2(min2max_vec[1], min2max_vec[0])
    print('test min2max_vec and yaw', min2max_vec, yaw)
    pitch = math.atan2(max_d-min_d, 0.023)
    if min2max_vec[0] > 0:
        yaw = math.pi/2 + yaw
    elif min2max_vec[0] < 0:
        if min2max_vec[1] > 0:
            yaw = -(3*math.pi/2-yaw)
        elif min2max_vec[1] < 0:
            yaw = -(-yaw-math.pi/2)

    if surf_normal == []:
        print("surf normal empty")
        pose={
            'x':position[0],
            'y':position[1],
            'z':m_c_dist,
            'yaw':yaw,
            'pitch':pitch,
            'normal':surf_normal
        }
    else:
        pose={
            'x':position[0],
            'y':position[1],
            'z':m_c_dist,
            'yaw':yaw,
            'pitch':pitch,
            'normal':surf_normal
        }
    return pose

In [ ]:
def sortFourth(val): 
    return val[3]  

def generate_stone_pose(depth_image, depth_array, seg_result):
    depth_copy = depth_image.copy()
    mask_size = []
    mask_store = 4
    window = 10

    # Rank masks by footprints
    for m in range(seg_result['masks'].shape[2]):
        mask = seg_result['masks'][:,:,m]
        #print(mask.shape)
        mask = mask.astype(np.uint8)
        mask_size.append(np.sum(mask))
    mask_index = np.argsort(np.array(mask_size))[-mask_store:seg_result['masks'].shape[2]]

    mask_center = []
    max_min_pair = []
    dist_min = []
    dist_max = []
    count_m = 0
    points = []
    point_show = []
    points_ero = []
    point_show_ero = []
    kernel = np.ones((15,15),np.uint8)
    
    for m in mask_index:
        mask = seg_result['masks'][:,:,m]
        mask = mask.astype(np.uint8)
        edges = cv2.Canny(mask,0,1)

        distance = []
        edge_point = []
        min_candidate = []
        max_candidate = []
        count = 0
        dist_max_ = -100000
        max_index = [0, 0]
        dist_min_ = 100000
        min_index = [0, 0]
        depth_intrin = [321.8862609863281, 238.18316650390625, 612.0938720703125, 611.785888671875] # cx, cy, fx, fy
        #print(depth_intrin)
        overall_mask_x = []
        overall_mask_y = []
        point = [] # store pointcloud of each go stone
        point_ero = [] # store pointcloud of each eroded go stone
        #erosion = cv2.erode(mask,kernel,iterations = 3)
        
        edge_pixel_set = np.argwhere(edges == 255)
        edge_pixel_set_copy=edge_pixel_set.tolist()
        edge_pixel_set_copy2=edge_pixel_set.tolist()
        for i in range(len(edge_pixel_set_copy2)):
            if edge_pixel_set_copy2[i][0]>=depth_copy.shape[0] or edge_pixel_set_copy2[i][1]>=depth_copy.shape[1]:
                edge_pixel_set_copy.remove(edge_pixel_set_copy2[i])
        edge_pixel_set=np.array(edge_pixel_set_copy)
        overall_mask_x = edge_pixel_set[:,1].tolist()
        overall_mask_y = edge_pixel_set[:,0].tolist()
        for index in edge_pixel_set:
            depth_copy[index[0],index[1]] = [0, 255, 0]
        edge_point = edge_pixel_set[:]
        edge_point[:, [0, 1]] = edge_point[:, [1, 0]]
        edge_point = edge_point.tolist()
        for edge_pixel in edge_pixel_set_copy:
            if depth_array[edge_pixel[0],edge_pixel[1]]/1000 > dist_max_ and depth_array[edge_pixel[0],edge_pixel[1]]/1000 != 0:
                dist_max_ = depth_array[edge_pixel[0],edge_pixel[1]]/1000
                max_index = [edge_pixel[1], edge_pixel[0]] 
            if depth_array[edge_pixel[0],edge_pixel[1]]/1000 < dist_min_ and depth_array[edge_pixel[0],edge_pixel[1]]/1000 != 0:
                dist_min_ = depth_array[edge_pixel[0],edge_pixel[1]]/1000
                min_index = [edge_pixel[1], edge_pixel[0]] 

        erosion = cv2.erode(mask,kernel,iterations = 3)
        
        mask_pixel_set = np.argwhere(mask == 1)
        mask_pixel_set_copy=mask_pixel_set.tolist()
        for i in mask_pixel_set:
            if i[0]>=depth_copy.shape[0] or i[1]>=depth_copy.shape[1]:
                mask_pixel_set_copy.remove(i.tolist())
        for mask_pixel in mask_pixel_set_copy:
            i = mask_pixel[0]
            j = mask_pixel[1]
            xy = pixel_to_camera([j, i], depth_intrin, depth_array[i, j])
            point.append([xy[0], xy[1], depth_array[i, j]]) # add pointcloud
        erosion_pixel_set = np.argwhere(erosion == 1)
        erosion_pixel_set_copy=erosion_pixel_set.tolist()
        for i in erosion_pixel_set:
            if i[0]>=depth_copy.shape[0] or i[1]>=depth_copy.shape[1]:
                erosion_pixel_set_copy.remove(i.tolist())
        for erosion_pixel in erosion_pixel_set_copy:
            i = erosion_pixel[0]
            j = erosion_pixel[1]
            xy = pixel_to_camera([j, i], depth_intrin, depth_array[i, j])
            point_ero.append([xy[0], xy[1], depth_array[i, j]]) # add pointcloud
       
        point_show = point_show + point # pointcloud for all go stone candidates
        points.append(point)
        point_show_ero = point_show_ero + point_ero # pointcloud for all eroded go stone candidates
        points_ero.append(point_ero)
        
        #print('another dist max min', dist_max_, dist_min_)
        cv2.circle(depth_copy,tuple(edge_point[0]), 5, (0,0,255), 2)

        max_min_pair.append([max_index, min_index])
        dist_max.append(dist_max_)
        dist_min.append(dist_min_)
        

        mask_center.append([int(round(np.mean(overall_mask_x))), int(round(np.mean(overall_mask_y))), count_m]) #in pixel frame
        print('mask center', mask_center[count_m][:2])
        
        cv2.circle(depth_copy,tuple(max_min_pair[count_m][0]), 5, (0,0,0), 2)
        cv2.circle(depth_copy,tuple(max_min_pair[count_m][1]), 5, (255,0,0), 2)
        cv2.circle(depth_copy,tuple(mask_center[count_m][:2]), 5, (255,0,0), 1)
        #print(get_mask_pose(depth_intrin, depth_array, mask_center[count_m], max_min_pair[mask_center[count_m][2]], dist_max[mask_center[count_m][2]], dist_min[mask_center[count_m][2]],[]))
        count_m = count_m + 1
    
    image_center = [depth_copy.shape[1]/2, depth_copy.shape[0]/2]
    for i in range(mask_store):
        mask_center[i].append(np.linalg.norm(np.array(mask_center[i][:2])-np.array(image_center)))
    mask_center.sort(key=sortFourth)
    plt.figure()
    plt.rcParams['figure.figsize'] = [24, 12]
    plt.imshow(depth_copy)
    
    # Generate surface normal
    pcd_ero = o3d.geometry.PointCloud()
    pcd_ero.points = o3d.utility.Vector3dVector(points_ero[mask_center[0][2]])
    pcd_temp = o3d.geometry.PointCloud()
    pcd_temp.points = o3d.utility.Vector3dVector(point_show_ero)
    o3d.io.write_point_cloud("test_ros_ero.ply", pcd_temp)
    o3d.io.write_point_cloud("test_ros_ero_selected.ply", pcd_ero)
    downpcd = o3d.geometry.voxel_down_sample(pcd_ero, voxel_size=1)
    o3d.geometry.estimate_normals(downpcd, search_param=o3d.geometry.KDTreeSearchParamHybrid(
        radius=10, max_nn=30))
    print("Normal as a numpy array")
    
    for i in range(np.asarray(downpcd.normals).shape[0]):
        if downpcd.normals[i][2] > 0:
            downpcd.normals[i][0] = -downpcd.normals[i][0]
            downpcd.normals[i][1] = -downpcd.normals[i][1]
            downpcd.normals[i][2] = -downpcd.normals[i][2]
    
    normals = np.asarray(downpcd.normals)
    surf_normal = np.sum(normals, axis=0) / normals.shape[0]
    
    return get_mask_pose(depth_intrin, depth_array, mask_center[0], max_min_pair[mask_center[0][2]], dist_max[mask_center[0][2]], dist_min[mask_center[0][2]], surf_normal), depth_copy

In [ ]:
is_detect = 0
img_index = 0
def img_index_callback(data):
    global img_index
    global is_detect
    
    print(data.data)
    img_index = data.data
    is_detect = 1

In [ ]:

rospy.Subscriber('/stone_img_index', String, img_index_callback)

In [ ]:
#is_detect = 1
import time

In [ ]:
while (True):
    if(is_detect == 1):
        
        pose_pub = rospy.Publisher('/stone_pose', stone_pose, queue_size=10)
        
        img_index = str(img_index)
              
        
        # instance segmentation
        image = scipy.misc.imread("JPEGImages/"+img_index+".jpeg")
        
        t = time.time()
        # Run detection
        results = model.detect([image], verbose=1)
        
        print('time:',time.time()-t)
        
        # Visualize results
        r = results[0]
        
        #depth_image = cv2.imread("depth/"+img_index+".jpeg")
        depth_image = cv2.imread("depth/"+img_index+".jpeg")
        depth_array = np.load("./depth/"+img_index+".npy")
        pose, depth_copy = generate_stone_pose(depth_image, depth_array, r)
        #plt.figure()
        #plt.rcParams['figure.figsize'] = [24, 12]
        #plt.imshow(depth_copy)
        
        visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                                    class_names, r['scores'])
        
        stone_pose_msg = stone_pose()
        stone_pose_msg.x = pose['x']
        stone_pose_msg.y = pose['y']
        stone_pose_msg.z = pose['z']
        stone_pose_msg.yaw = pose['yaw']
        stone_pose_msg.pitch = pose['pitch']
        stone_pose_msg.normal = pose['normal']

        pose_pub.publish(stone_pose_msg)
        print('stone_pose_msg is ', stone_pose_msg)
        
        is_detect = 0